# A7 - Data Processing

This notebook handles basic clean up and pre-processing of all the raw input data. The output will later be used for visualizations and regression analysis.

In [1]:
import pandas as pd
import numpy as np

Renaming columns to more intuitive names, dropping unnecessary columns, and performing some type conversions.

In [2]:
def load_table(file_name, metric_name):
    table = pd.read_csv(file_name)
    table = table.rename(columns = {'Province_State': 'State', 'Admin2': 'County', 'Lat': 'Latitude', 'Long_': 'Longitude'})
    table = table.loc[(table['iso3'] == 'USA') & (table['State'] != "Grand Princess") & (table['State'] != "Diamond Princess") & (table['Latitude'] != 0.0) & (table['Longitude'] != 0.0)]
    table = table.drop(['UID', 'iso2', 'iso3', 'Combined_Key', 'code3', 'Country_Region'], axis=1)
    keys = ['State', 'County', 'FIPS', 'Latitude', 'Longitude']
    if 'Population' in table.columns:
        keys.append('Population')
    table = table.melt(id_vars=keys, var_name='Date', value_name=metric_name).reset_index()
    table['Date'] = pd.to_datetime(table['Date'], infer_datetime_format=True)
    table['FIPS'] = table['FIPS'].astype("Int64")
    table = table.drop(['index'], axis=1)
    
    return table

Loading the record of confirmed cases and deaths per county [1]. 

In [3]:
cases = load_table(file_name='cases/RAW_us_confirmed_cases.csv', metric_name='Cases')

In [4]:
deaths = load_table(file_name='cases/RAW_us_deaths.csv', metric_name='Deaths')

In [5]:
cases_and_deaths = cases.merge(deaths, on=['State', 'County', 'FIPS', 'Latitude', 'Longitude', 'Date'])

While the source dataset continues to evolve, at the time of this project, the available data included records from January 2020 to November 2021.

In [6]:
cases_and_deaths.Date.min(), cases_and_deaths.Date.max()

(Timestamp('2020-01-22 00:00:00'), Timestamp('2021-11-01 00:00:00'))

## Rates by State

Calculating the aggregated counts of deaths and population by state.

In [7]:
rates_by_state = cases_and_deaths[['State', 'Cases', 'Deaths', 'Population']].groupby(by='State').sum()

Calculating the death rate per 1,000 population, and the infection mortality rate per 1,000 cases.

In [8]:
rates_by_state['Deaths by 1,000 Population'] = 1000 * rates_by_state['Deaths'] / rates_by_state['Population']
rates_by_state['Deaths by 1,000 Cases'] = 1000 * rates_by_state['Deaths'] / rates_by_state['Cases']

In [9]:
rates_by_state.head(10)

,Cases,Deaths,Population,"Deaths by 1,000 Population","Deaths by 1,000 Cases"
State,,,,,
Alabama,209071434,3882780,3187070250,1.218291,18.571547
Alaska,25934080,130626,481646750,0.271207,5.036847
Arizona,320899015,6283173,4731166050,1.328039,19.579907
Arkansas,129823216,2128392,1961572600,1.085044,16.394541
California,1379627257,20898866,25682944950,0.813725,15.148197
Colorado,185631795,2544127,3743178400,0.679670,13.705233
Connecticut,117614131,3559254,2317436550,1.535858,30.262129
Delaware,37168579,663011,632946600,1.047499,17.837943
District of Columbia,18392671,477832,458736850,1.041625,25.979478


Saving the output for further analysis and visualizations.

In [10]:
rates_by_state.reset_index().to_csv('mortality_rates_by_state.csv')

## Rates by County

Aggregating the daily records as a snapshot per county.

In [11]:
rates_by_county = cases_and_deaths[['State', 'County', 'Cases', 'Deaths', 'Population']].groupby(by=['State', 'County']).sum()

In [12]:
rates_by_county['Deaths by 1,000 Population'] = 1000 * rates_by_county['Deaths'] / rates_by_county['Population']
rates_by_county['Deaths by 1,000 Case'] = 1000 * rates_by_county['Deaths'] / rates_by_county['Cases']

In [13]:
rates_by_county.head(10)

Cases  Deaths  Population  Deaths by 1,000 Population  \
State   County                                                              
Alabama Autauga   2610739   37993    36314850                    1.046211   
        Baldwin   8588525  113444   145102100                    0.781822   
        Barbour    937910   19576    16045900                    1.220000   
        Bibb      1031720   22351    14556100                    1.535507   
        Blount    2559096   44830    37586900                    1.192703   
        Bullock    502704   14972     6565650                    2.280353   
        Butler     916640   29780    12641200                    2.355789   
        Calhoun   5467623  109423    73843250                    1.481828   
        Chambers  1468175   45343    21615100                    2.097746   
        Cherokee   747609   16367    17027400                    0.961215   

                  Deaths by 1,000 Case  
State   County                          
Alabama Autauga              14.552585  
        Baldwin              13.208787  
        Barbour              20.871939  
        Bibb                 21.663824  
        Blount               17.517905  
        Bullock              29.782934  
        Butler               32.488218  
        Calhoun              20.012901  
        Chambers             30.883921  
        Cherokee             21.892460

Saving the rates by county for further analysis and visualizations.

In [14]:
rates_by_county.to_csv('mortality_rates_by_county.csv')

## Health Infrastructure Metrics

In [15]:
def load_health_infrastructure_data(file_name):
    return pd.read_csv(f"health_infrastructure/{file_name}").rename({'Location': 'State'}, axis=1)

### Hospitals

The total number of hospitals [2] per state.

In [16]:
total_hospitals = load_health_infrastructure_data("total_hospitals.csv")

In [17]:
all_metrics = rates_by_state.merge(total_hospitals, left_on='State', right_on='State', how='left')
all_metrics['Hospitals per 1,000 Population'] = 1000 * all_metrics['Total Hospitals'] / all_metrics['Population']

### Community Health Centers

The number of community health centers [3] per state.

In [18]:
health_centers = load_health_infrastructure_data("community_health_centers.csv")

In [19]:
all_metrics = all_metrics.merge(health_centers, left_on='State', right_on='State', how='left')
all_metrics['Community Health Centers per 1,000 Population'] = 1000 * all_metrics['Total CHCs'] / all_metrics['Population']

### Hospital Beds

Number of hospital beds [4] per state. Notice this dataset doesn't contain totals, but a rate of number of beds per 1,000 population.

In [20]:
hospital_beds = load_health_infrastructure_data("hospital_beds_per_1000_people.csv")

In [21]:
all_metrics = all_metrics.merge(hospital_beds, left_on='State', right_on='State', how='left')
all_metrics['Total beds per 1,000 Population'] = all_metrics['Total']

### ICU Beds

Number of ICU beds [5] per state.

In [22]:
icu_beds = load_health_infrastructure_data("icu_beds.csv")

In [23]:
all_metrics = all_metrics.merge(icu_beds, left_on='State', right_on='State', how='left')
all_metrics['ICU beds per 1,000 Population'] = 1000 * all_metrics['ICU Beds'] / all_metrics['Population']

### Primary Care

Number of professionally active primary care physicians [6] per state.

In [24]:
primary_care = load_health_infrastructure_data("primary_care.csv")

In [25]:
all_metrics = all_metrics.merge(primary_care, left_on='State', right_on='State', how='left')
all_metrics['Total Primary Care Practitioners per 1,000 Population'] = 1000 * all_metrics['Total Primary Care'] / all_metrics['Population']

### Primary Care Shortage

An index indicating the degree of shortage in primary care providers [7] per state.

Per documentation: "Percent of Need Met" is computed by dividing the number of physicians available to serve the population of the area, group, or facility by the number of physicians that would be necessary to eliminate the primary are HPSA (based on a ratio of 3,500 to 1 (3,000 to 1 where high needs are indicated)).

In [26]:
primary_care_shortage = load_health_infrastructure_data("primary_care_shortage.csv")

In [27]:
all_metrics = all_metrics.merge(primary_care_shortage, left_on='State', right_on='State', how='left')
all_metrics['Percentage of Practitioners Needed Met'] = all_metrics['Percent of Need Met']

## Output

In [28]:
all_metrics.to_csv("all_metrics.csv")

## Data Sources

[1] Record of COVID-19 confirmed cases and deaths by state, dataset from Johns Hopkins University and distributed through Kaggle under CC BY 4.0 license. Available at https://www.kaggle.com/antgoldbloom/covid19-data-from-john-hopkins-university

[2] “Total Hospitals”, 2019, AHA Annual Survey, Copyright 2020 by Health Forum, LLC, an affiliate of the American Hospital Association. Special data request, 2020. Available from KFF State Health Facts, https://www.kff.org/other/state-indicator/total-hospitals/

[3] “Community Health Center Delivery Sites and Patient Visits”, 2019, George Washington University analysis of the 2020 Uniform Data System, Health Resources and Services Administration. Special Data Request, September 2021. Available from KFF State Health Facts, https://www.kff.org/other/state-indicator/community-health-center-sites-and-visits/

[4] “Hospital Beds per 1,000 Population by Ownership Type”, 2019, AHA Annual Survey, Copyright 2020 by Health Forum, LLC, an affiliate of the American Hospital Association. Special data request, 2020. Available from KFF State Health Facts, https://www.kff.org/other/state-indicator/beds-by-ownership

[5] “ICU Beds”, 2018, KFF analysis of merged American Hospital Directory and 2018 AHA Annual Survey data. Available from KFF State Health Facts, https://www.kff.org/other/state-indicator/icu-beds/

[6] “Professionally Active Primary Care Physicians by Field”, 2021, Special data request on State Licensing Information from Redi-Data, Inc., June 202. Available from KFF State Health Facts, https://www.kff.org/other/state-indicator/primary-care-physicians-by-field/

[7] “Primary Care Health Professional Shortage Areas (HPSAs)”, 2021, Bureau of Health Workforce, Health Resources and Services Administration (HRSA), U.S. Department of Health & Human Services, Designated Health Professional Shortage Areas Statistics: Designated HPSA Quarterly Summary, as of September 30, 2021 available at https://data.hrsa.gov/topics/health-workforce/shortage-areas and obtained from KFF State Health Facts, https://www.kff.org/other/state-indicator/primary-care-health-professional-shortage-areas-hpsas/